# Tutorial 1: solving the power-flow equations with ExaModels

In this tutorial, we detail how to use ExaModels to solve the power flow
equations on the GPU. We start by describing the model we use, and then write
a basic Newton solver in Julia. Then we detail how to port the algorithm to the GPU
for faster performance

We start by importing the usual packages (including JLD2, a package to import
serialized data in Julia)

In [1]:
using LinearAlgebra
using SparseArrays

using NLPModels
using ExaModels

using JLD2

include("utils.jl")

get_block_reordering (generic function with 1 method)

We load the classical case9ieee instance, here generated using the MATPOWER
file found in [matpower repo](https://github.com/MATPOWER/).

In [2]:
DATA_DIR = "/home/fpacaud/dev/examodels-tutorials/instances"
data = JLD2.load(joinpath(DATA_DIR, "case9.jld2"))["data"]

(bus = @NamedTuple{i::Int64, pd::Float64, gs::Float64, qd::Float64, bs::Float64, bus_type::Int64}[(i = 1, pd = 0.9, gs = 0.0, qd = 0.3, bs = 0.0, bus_type = 1), (i = 2, pd = 0.0, gs = 0.0, qd = 0.0, bs = 0.0, bus_type = 1), (i = 3, pd = 0.0, gs = 0.0, qd = 0.0, bs = 0.0, bus_type = 1), (i = 4, pd = 1.0, gs = 0.0, qd = 0.35, bs = 0.0, bus_type = 1), (i = 5, pd = 0.0, gs = 0.0, qd = 0.0, bs = 0.0, bus_type = 2), (i = 6, pd = 1.25, gs = 0.0, qd = 0.5, bs = 0.0, bus_type = 1), (i = 7, pd = 0.0, gs = 0.0, qd = 0.0, bs = 0.0, bus_type = 1), (i = 8, pd = 0.0, gs = 0.0, qd = 0.0, bs = 0.0, bus_type = 2), (i = 9, pd = 0.0, gs = 0.0, qd = 0.0, bs = 0.0, bus_type = 3)], gen = @NamedTuple{i::Int64, cost1::Float64, cost2::Float64, cost3::Float64, bus::Int64}[(i = 1, cost1 = 850.0000000000001, cost2 = 120.0, cost3 = 600.0, bus = 5), (i = 2, cost1 = 1225.0, cost2 = 100.0, cost3 = 335.0, bus = 8), (i = 3, cost1 = 1100.0, cost2 = 500.0, cost3 = 150.0, bus = 9)], arc = @NamedTuple{i::Int64, rate_a::Floa

The number of buses, generators and lines are:

In [3]:
nbus = length(data.bus)
ngen = length(data.gen)
nlines = length(data.branch)

9

We load the indexes of the PV buses and the generators at the PV buses:

In [4]:
pv_buses = get_pv_buses(data)
free_gen = get_free_generators(data)

2-element Vector{Int64}:
 1
 2

## Implementing the power flow equations with ExaModels

When using the polar formulation, the power flow model requires the following variables:

1. The voltage magnitude at nodes $v_m$
2. The voltage angles at nodes $v_a$
3. The active power generation $p_g$
4. The reactive power generation $q_g$
5. The active power flow through the lines $p$
6. The reactive power flow through the lines $q$

The variables $p$ and $q$ are dependent variables depending on the voltage magnitudes
and angles at the adjacent nodes. The structure of the problem implies that the only
degree-of-freedom are the voltage magnitude at the PV and REF buses, the voltage angle at the REF buses
(usually set equal to 0) and the active power generation at the PV buses.

We define the variable in ExaModels.

In [5]:
core = ExaCore()
va = variable(core, nbus)
vm = variable(core, nbus; start = data.vm0)
pg = variable(core, ngen; start=data.pg0)
qg = variable(core, ngen; start=data.qg0)
p = variable(core, 2*nlines) # FR and TO lines
q = variable(core, 2*nlines) # FR and TO lines

Variable

  x ∈ R^{18}


We set the initial values in `vm`, `pg` and `qg` using the setpoint values
specified in the matpower file.

As we solve the power flow equations, the degree-of-freedom are fixed. We fix them
in the model using a set of equality constraints:
We iterate over the reference buses to set their voltage and to 0

In [6]:
c1 = constraint(core, va[i] for i in data.ref_buses)

Constraint

  s.t. (...)
       g♭ ≤ [g(x,p)]_{p ∈ P} ≤ g♯

  where |P| = 1


and over the PV buses to set the voltage magnitude to the setpoint

In [7]:
c01 = constraint(core, vm[i] for i in pv_buses; lcon=data.vm0[pv_buses], ucon=data.vm0[pv_buses])

Constraint

  s.t. (...)
       g♭ ≤ [g(x,p)]_{p ∈ P} ≤ g♯

  where |P| = 3


and finally over the generators to fix the active power generation (except at the REF buses):

In [8]:
c02 = constraint(core, pg[i] for i in free_gen; lcon=data.pg0[free_gen], ucon=data.pg0[free_gen])

Constraint

  s.t. (...)
       g♭ ≤ [g(x,p)]_{p ∈ P} ≤ g♯

  where |P| = 2


We use the same model as in [MATPOWER](https://matpower.org/docs/manual.pdf)
to model the transmission lines, based on the standard $π$ transmission line model in series with an ideal phase-shifting transformer.
Using the polar formulation, the active power through the line $(i, j)$ is defined as
$$
  p_{i j} = g_{i i} v_{m,i}^2
  + g_{i j} v_{m, i} v_{m, j} \cos(v_{a, i} - v_{a, j})
  + b_{i j} v_{m, i} v_{m, j} \sin(v_{a, i} - v_{a, j})

$$
and the reactive power is defined similarly as
$$
  q_{i j} = g_{i i} v_{m,i}^2
  + g_{i j} v_{m, i} v_{m, j} \sin(v_{a, i} - v_{a, j})
  - b_{i j} v_{m, i} v_{m, j} \cos(v_{a, i} - v_{a, j})

$$
Using ExaModels, these two equations translate to the following constraints at the origin

In [9]:
c2 = ExaModels.constraint(
    core,
    p[b.f_idx] - b.c5 * vm[b.f_bus]^2 -
    b.c3 * (vm[b.f_bus] * vm[b.t_bus] * cos(va[b.f_bus] - va[b.t_bus])) -
    b.c4 * (vm[b.f_bus] * vm[b.t_bus] * sin(va[b.f_bus] - va[b.t_bus])) for
    b in data.branch
)
c3 = ExaModels.constraint(
    core,
    q[b.f_idx] +
    b.c6 * vm[b.f_bus]^2 +
    b.c4 * (vm[b.f_bus] * vm[b.t_bus] * cos(va[b.f_bus] - va[b.t_bus])) -
    b.c3 * (vm[b.f_bus] * vm[b.t_bus] * sin(va[b.f_bus] - va[b.t_bus])) for
    b in data.branch
)

Constraint

  s.t. (...)
       g♭ ≤ [g(x,p)]_{p ∈ P} ≤ g♯

  where |P| = 9


Similarly, the power flow at the destination are

In [10]:
c4 = ExaModels.constraint(
    core,
    p[b.t_idx] - b.c7 * vm[b.t_bus]^2 -
    b.c1 * (vm[b.t_bus] * vm[b.f_bus] * cos(va[b.t_bus] - va[b.f_bus])) -
    b.c2 * (vm[b.t_bus] * vm[b.f_bus] * sin(va[b.t_bus] - va[b.f_bus])) for
    b in data.branch
)

Constraint

  s.t. (...)
       g♭ ≤ [g(x,p)]_{p ∈ P} ≤ g♯

  where |P| = 9


Reactive power flow, TO

In [11]:
c5 = ExaModels.constraint(
    core,
    q[b.t_idx] +
    b.c8 * vm[b.t_bus]^2 +
    b.c2 * (vm[b.t_bus] * vm[b.f_bus] * cos(va[b.t_bus] - va[b.f_bus])) -
    b.c1 * (vm[b.t_bus] * vm[b.f_bus] * sin(va[b.t_bus] - va[b.f_bus])) for
    b in data.branch
)

Constraint

  s.t. (...)
       g♭ ≤ [g(x,p)]_{p ∈ P} ≤ g♯

  where |P| = 9


It remains to write the power flow balance equations at each bus.
They are defined, for the active power flow at bus $i$
$$
   p_{g, i} - p_{d, i} - g_{s,i} v_{m,i}^2 = ∑_{j ∈ N(i)} p_{ij}

$$
and for the reactive power flo at bus $i$
$$
   q_{g, i} - q_{d, i} - b_{s,i} v_{m,i}^2 = ∑_{j ∈ N(i)} q_{ij}

$$

Note that both set of constraints require to sum over the power flow at the adjacent lines.
As we have seen before, ExaModels defines the sum with a reduction over a given iterator.
As a consequence, we will evaluate the first terms $p_{g, i} - p_{d, i} - g_{s,i} v_{m,i}^2 $
apart from the sum $∑_{j ∈ N(i)} p_{ij}$ in the expression tree defining the active
power flow balance. This translates to the following syntax in ExaModels. We first
iterate over all the buses to define the first part of the expressions:

In [12]:
active_flow_balance = ExaModels.constraint(core, b.pd + b.gs * vm[b.i]^2 for b in data.bus)

Constraint

  s.t. (...)
       g♭ ≤ [g(x,p)]_{p ∈ P} ≤ g♯

  where |P| = 9


Then we modify the constraint inplace to add the contribution of the adjacent lines

In [13]:
ExaModels.constraint!(core, active_flow_balance, a.bus => p[a.i] for a in data.arc)

Constraint Augmentation

  s.t. (...)
       g♭ ≤ (...) + ∑_{p ∈ P} h(x,p) ≤ g♯

  where |P| = 18


and finally, we add the contribution of the generators connected to each bus:

In [14]:
ExaModels.constraint!(core, active_flow_balance, g.bus => -pg[g.i] for g in data.gen)

Constraint Augmentation

  s.t. (...)
       g♭ ≤ (...) + ∑_{p ∈ P} h(x,p) ≤ g♯

  where |P| = 3


We follow the same procedure for the reactive power flow balance:

In [15]:
reactive_flow_balance = ExaModels.constraint(core, b.qd - b.bs * vm[b.i]^2 for b in data.bus)
ExaModels.constraint!(core, reactive_flow_balance, a.bus => q[a.i] for a in data.arc)
ExaModels.constraint!(core, reactive_flow_balance, g.bus => -qg[g.i] for g in data.gen)

Constraint Augmentation

  s.t. (...)
       g♭ ≤ (...) + ∑_{p ∈ P} h(x,p) ≤ g♯

  where |P| = 3


We have now defined all the equations needed to evaluate the power flow equations!
Note that we have defined all the expressions inside ExaModels: to evaluate them,
we convert the ExaCore to a proper ExaModel as:

In [16]:
nlp = ExaModel(core)

An ExaModel{Float64, Vector{Float64}, ...}

  Problem name: Generic
   All variables: ████████████████████ 60     All constraints: ████████████████████ 60    
            free: ████████████████████ 60                free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 60    
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: ( 79.34% sparsity)   378             linear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                    nonlinear: ████████████████████ 60    
                                                         nnzj: ( 93.17% sparsity)   246   



Using NLPModels, evaluating the power flow at the initial setpoint amounts to

In [17]:
x0 = NLPModels.get_x0(nlp)
c = NLPModels.cons(nlp, x0)

60-element Vector{Float64}:
  0.0
  1.0
  1.0
  1.0
  1.63
  0.85
  0.0
  0.0
  0.0
  0.0
  ⋮
  0.3
  0.0
  0.0
  0.35
 -0.0654
  0.5
  0.0
  0.10949999999999999
 -0.2703

Remember that the first equations `c1`, `c01`, and `c02` are fixing the degree-of-freedom
to their setpoint. The power flow equations per-se are defined by the remaining equations,
starting with the constraint `c2`:

In [18]:
m_fixed = c2.offset # use offset to determine where does the power flow eq. start in the model

6

We compute the norm-2 of the initial residual:

In [19]:
residual = norm(c[m_fixed+1:end])

2.8281211961300383

Note that if the power flow equations are satisfied, this residual should be 0, which is not
the case here. We remember that our degree-of-freedom are:

- voltage angle at ref buses;
- voltage magnitude at PV and ref buses;
- active power generation at PV buses;

We keep the degree-of-freedom fixed, and looks for the dependent variables
satisfying the power flow equations for this given setpoint. To do this, we will
use Newton method over the power flow balance equations.

## Solving the power flow equations using the Newton algorithm

We load the numbers of variables, constraints and nonzeros in the Jacobian
(all these values are provided automatically by ExaModels):

In [20]:
n = NLPModels.get_nvar(nlp)
m = NLPModels.get_ncon(nlp)
nnzj = NLPModels.get_nnzj(nlp)

246

We load the index of the degree-of-freedom in our model using a utility function:

In [21]:
ind_dof = get_index_dof(data)

6-element Vector{Int64}:
  9
 14
 17
 18
 19
 20

and the indexes of dependent variables are automatically defined as

In [22]:
ind_dep = setdiff(1:n, ind_dof)

54-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
 10
 11
  ⋮
 52
 53
 54
 55
 56
 57
 58
 59
 60

We start by evaluating the Jacobian of our model using NLPModels syntax.
We get the sparsity pattern of our Jacobian in COO format directly by using:

In [23]:
Ji = similar(x0, Int, nnzj)
Jj = similar(x0, Int, nnzj)
NLPModels.jac_structure!(nlp, Ji, Jj)

([1, 2, 3, 4, 5, 6, 7, 7, 7, 7  …  58, 56, 52, 53, 57, 54, 53, 56, 59, 60], [9, 14, 17, 18, 19, 20, 25, 12, 13, 3  …  54, 55, 56, 57, 58, 59, 60, 22, 23, 24])

and we evaluate the nonzero values using

In [24]:
Jx = similar(x0, nnzj)
NLPModels.jac_coord!(nlp, x0, Jx)

246-element Vector{Float64}:
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
 -1.1550874808900968
  1.1550874808900968
 -9.784270426363172
  ⋮
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
 -1.0
 -1.0
 -1.0

Julia uses the CSC format by default to store sparse matrix. We can convert
our Jacobian to CSC directly using Julia syntax:

In [25]:
J = sparse(Ji, Jj, Jx, m, n)

60×60 SparseArrays.SparseMatrixCSC{Float64, Int64} with 246 stored entries:
⎡⠀⠀⠀⠀⠁⠀⠐⠀⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⡤⠀⢀⠀⢠⠄⠀⡀⠑⠀⠀⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢤⠈⢂⠃⠠⡄⠑⡘⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠢⣂⠈⠁⠔⢔⡀⠉⠠⠀⠀⠀⠀⠀⠀⠑⠄⠀⠀⠀⠀⡀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣀⠑⠄⠎⢀⡈⠢⠰⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠢⡀⠀⠀⠀⠀⠀⠀⎥
⎢⢌⠄⠘⠂⡠⡡⠀⠓⢀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠢⡀⠀⠀⠀⠀⎥
⎢⠀⠫⡀⡔⠀⠘⢅⢠⠂⠀⠀⠀⠀⠀⠀⠀⠈⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⡙⡀⠰⠄⢈⢃⠀⠦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠈⢖⠀⡠⠁⠱⡂⢀⠌⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠀⠀⠐⢄⠀⠀⠀⎥
⎢⠲⠀⢡⡁⠐⠆⠈⣌⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⎥
⎢⠑⠁⠀⠀⠪⡊⠀⠀⠐⠀⠀⠀⠀⠀⡀⠠⠀⠀⠠⡀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠑⎥
⎢⠀⠀⠀⠀⠀⠈⠢⡀⠀⠄⠀⠀⠁⠂⢀⠀⠐⠁⠄⢀⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⢀⠀⠀⠈⠢⠐⠄⠀⠐⠈⠀⠁⠄⠈⠀⠀⠀⠀⠀⠀⢀⠀⠀⢀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠂⠄⠁⠀⠠⠂⡀⠁⠊⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⠐⠈⠂⡀⠐⠀⠈⠀⎦

And we can extract from the Jacobian the part associated to the power flow balance;

In [26]:
G = J[m_fixed+1:end, ind_dep]

54×54 SparseArrays.SparseMatrixCSC{Float64, Int64} with 216 stored entries:
⎡⠀⠫⡀⡔⠀⠫⢠⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⡙⡀⠰⠄⡙⡀⠦⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠈⢖⠀⡠⠈⢖⢀⠀⠀⠀⠀⠀⠀⠁⠀⠀⠀⠀⠢⡀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠲⠀⢡⡁⠲⠀⣌⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠢⡀⠀⠀⠀⠀⠀⎥
⎢⠑⡥⠀⢀⠑⡥⠀⠀⠀⠀⠀⠀⠀⠠⡀⠀⠀⠀⠀⠀⠀⠈⠂⠀⠀⠀⠀⎥
⎢⢤⠈⢂⠃⢤⠈⡘⠀⠀⠀⠀⠀⠀⠀⠈⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠢⣂⠈⠁⠢⣂⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠢⠀⠀⠀⠀⢀⠀⠀⠀⠀⎥
⎢⣀⠑⠄⠎⣀⠑⠰⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⎥
⎢⢌⠄⠘⠂⢌⠄⠓⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⎥
⎢⠀⠀⠀⠀⠑⢄⠀⠀⠀⠄⡀⠂⠈⢀⠄⠈⠂⠔⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠢⠀⠀⢀⠠⠐⠄⠀⠠⠁⠐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠢⡀⠀⠁⠀⠀⠀⠀⠀⠁⠀⠀⠀⠀⡀⠀⠄⠐⠀⡀⠐⠄⡠⎥
⎢⠀⠀⠀⠀⠀⠈⢄⠐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢁⠠⡀⠈⢀⠂⠠⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠑⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠀⠂⠀⠀⠀⠀⎦

This is the matrix we need in the Newton algorithm. But before implementing it, we need
one last routine to pass the data from the vector `Jx` (in COO format) to the nonzeros
in the CSC matrix G. To do this, we use the following trick:

In [27]:
Jx .= 1:nnzj # store index of each coefficient in Jx
J = sparse(Ji, Jj, Jx, m, n)  # convert the COO matrix to CSC
G = J[m_fixed+1:end, ind_dep] # extract the submatrix associated to the power flow equations
coo_to_csc = convert.(Int, nonzeros(G))

216-element Vector{Int64}:
  31
  45
  76
  90
 120
 136
 165
 181
  30
  36
   ⋮
 239
 167
 240
 172
 241
 177
 242
 182
 243

Using this vector of indices, we can automatically pass the data from Jx to G with:

In [28]:
nonzeros(G) .= Jx[coo_to_csc]

216-element Vector{Float64}:
  31.0
  45.0
  76.0
  90.0
 120.0
 136.0
 165.0
 181.0
  30.0
  36.0
   ⋮
 239.0
 167.0
 240.0
 172.0
 241.0
 177.0
 242.0
 182.0
 243.0

We are now in place to solve the power flow equations. We start by importing KLU:

In [29]:
using KLU

and we initialize the Newton algorithm by evaluating the model at the initial point:

In [30]:
x = copy(x0)
c = similar(x0, m)
d = similar(x0, length(ind_dep))     # descent direction
residual = view(c, m_fixed+1:m)      # get subvector associated to the power flow residual

NLPModels.cons!(nlp, x, c)
NLPModels.jac_coord!(nlp, x, Jx)
nonzeros(G) .= Jx[coo_to_csc]

216-element Vector{Float64}:
  10.51068205186793
  -5.588244962361526
  -1.9421912487147264
   1.2820091384241148
 -10.51068205186793
   5.588244962361526
   1.9421912487147264
  -1.2820091384241148
 -10.51068205186793
  11.604095563139932
   ⋮
   1.0
   1.0
   1.0
   1.0
   1.0
   1.0
   1.0
   1.0
   1.0

We compute the symbolic factorization using the direct solver KLU directly as

In [31]:
ls = klu(G)

KLU.KLUFactorization{Float64, Int64}
L factor:
54×54 SparseArrays.SparseMatrixCSC{Float64, Int64} with 148 stored entries:
⎡⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠤⢀⡱⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢑⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⡀⠀⠀⠀⣀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠀⠉⠁⢐⢒⢚⣳⣄⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠒⠒⢂⠉⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢈⣉⣳⣄⠀⠀⠀⎥
⎢⠀⠀⠀⠀⢁⡈⠂⠐⠂⠀⠀⠀⠀⠀⠀⣉⠀⠀⠀⢈⣉⣉⣿⣿⣷⣄⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⎦
U factor:
54×54 SparseArrays.SparseMatrixCSC{Float64, Int64} with 208 stored entries:
⎡⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠑⢎⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢠⠀⣧⢠⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠑⢎⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠨⡀⢼⠢⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠑⢤⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⢀⡉⢸⠈⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢆⠀⠀⠀⣤⠀⠀⠀⢠⡎⠈⠁⠈⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢄⡈⣷⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢿⠀⠀⠀⢠⡄⠀⠀⠀⡇⢸⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢦⢐⠈⢹⢨⡀⢀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢵⣾⢱⠀⠀⡀⢀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢾⡆⢰⡇⢸⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢾⣿⣾⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

The Newton algorithm writes:

In [32]:
max_iter = 10
tol = 1e-8

@info "Solving the power flow equations with Newton"
for i in 1:max_iter
    @info "It: $(i) residual: $(norm(residual))"
    if norm(residual) <= tol
        break
    end
    NLPModels.jac_coord!(nlp, x, Jx) # Update values in Jacobian
    nonzeros(G) .= Jx[coo_to_csc]
    klu!(ls, G)                      # Update numerical factorization
    ldiv!(d, ls, residual)           # Compute Newton direction using a backsolve
    x[ind_dep] .-= d
    NLPModels.cons!(nlp, x, c)
end

[ Info: Solving the power flow equations with Newton
[ Info: It: 1 residual: 2.8281211961300383
[ Info: It: 2 residual: 0.16641541672388815
[ Info: It: 3 residual: 0.003123124093240259
[ Info: It: 4 residual: 7.791517827071406e-7
[ Info: It: 5 residual: 6.401499746556268e-14


We observe that the Newton algorithm has converged in 5 iterations! The final
residual is not exactly 0 but is close enough (close to 1e-14).
We can recover the solution directly by looking at the values in the vector `x`:

In [33]:
va_sol = x[1:nbus]
vm_sol = x[nbus+1:2*nbus]

9-element Vector{Float64}:
 0.9754721770850534
 0.9870068523919056
 1.0033754364528011
 0.9856448817249474
 1.0
 0.9576210404299045
 0.9961852458090705
 1.0
 1.0

We implement the generation of the model in a function `powerflow_model`,
and the Newton algorithm in another function `solve_power_flow`:

In [34]:
include("powerflow.jl")

analyse_sparsity (generic function with 2 methods)

You can test the performance of Newton on various cases using the following code:

In [35]:
data = JLD2.load(joinpath(DATA_DIR, "pglib_opf_case1354_pegase.jld2"))["data"]
ngen = length(data.gen)
nbus = length(data.bus)
nlines = length(data.branch)

nlp = powerflow_model(data)
results = solve_power_flow(nlp)
nothing

[ Info: It: 1 residual: 114.94453623771956
[ Info: It: 2 residual: 9.77587903056736
[ Info: It: 3 residual: 0.7405168825291231
[ Info: It: 4 residual: 0.006184429522506654
[ Info: It: 5 residual: 1.721686601775372e-6
[ Info: It: 6 residual: 6.625913959067062e-12


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*